In [1]:
from sklearn import tree
import pandas as pd
import os

  ## Importing and cleaning the training dataset

In [2]:
data = pd.read_csv('../../Resources/bio_vars_frame.csv')
data.head()

,Unnamed: 0,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,...,bio_11,bio_12,bio_13,bio_14,bio_15,bio_16,bio_17,bio_18,bio_19,raster_frame
0,1,96.0,105.0,40.0,5390.0,250.0,-7.0,257.0,35.0,165.0,...,30.0,2184.0,350.0,25.0,61.0,1008.0,144.0,147.0,951.0,0
1,2,94.0,105.0,40.0,5385.0,249.0,-9.0,258.0,34.0,163.0,...,28.0,2196.0,354.0,24.0,62.0,1019.0,142.0,146.0,961.0,0
2,3,96.0,105.0,40.0,5407.0,251.0,-7.0,258.0,35.0,165.0,...,30.0,2162.0,348.0,24.0,62.0,1002.0,140.0,144.0,945.0,0
3,4,97.0,105.0,40.0,5437.0,251.0,-7.0,258.0,35.0,166.0,...,30.0,2144.0,345.0,24.0,62.0,993.0,139.0,143.0,937.0,0
4,5,93.0,105.0,40.0,5438.0,248.0,-11.0,259.0,32.0,163.0,...,27.0,2188.0,357.0,24.0,63.0,1023.0,138.0,143.0,966.0,0


In [3]:
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.dropna(axis = 0, how = 'any', inplace = True)
data.rename(columns={'bio_1': 'Annual Mean Temperature', 
                     'bio_2': 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
                     'bio_3': 'Isothermality',
                     'bio_4': 'Temperature Seasonality',
                     'bio_5': 'Max Temperature of Warmest Month',
                     'bio_6': 'Min Temperature of Coldest Month',
                     'bio_7': 'Temperature Annual Range',
                     'bio_8': 'Mean Temperature of Wettest Quarter',
                     'bio_9': 'Mean Temperature of Driest Quarter',
                     'bio_10': 'Mean Temperature of Warmest Quarter',
                     'bio_11': 'Mean Temperature of Coldest Quarter',
                     'bio_12': 'Annual Precipitation',
                     'bio_13': 'Precipitation of Wettest Month',
                     'bio_14': 'Precipitation of Driest Month',
                     'bio_15': 'Precipitation Seasonality (Coefficient of Variation)',
                     'bio_16': 'Precipitation of Wettest Quarter',
                     'bio_17': 'Precipitation of Driest Quarter',
                     'bio_18': 'Precipitation of Warmest Quarter',
                     'bio_19': 'Precipitation of Coldest Quarter',
                    }, inplace=True)

data = data[['raster_frame','Precipitation of Driest Month', 'Temperature Seasonality', 'Precipitation of Driest Quarter',
             'Annual Precipitation', 'Precipitation of Warmest Quarter']]

data.head()

,raster_frame,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,0,25.0,5390.0,144.0,2184.0,147.0
1,0,24.0,5385.0,142.0,2196.0,146.0
2,0,24.0,5407.0,140.0,2162.0,144.0
3,0,24.0,5437.0,139.0,2144.0,143.0
4,0,24.0,5438.0,138.0,2188.0,143.0


## Preprocessing the data for model training

In [4]:
target = data['raster_frame']
target_names = ["negative", "positive"]

In [5]:
X = data.drop(["raster_frame"], axis=1)
feature_names = X.columns
X.head()

,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,25.0,5390.0,144.0,2184.0,147.0
1,24.0,5385.0,142.0,2196.0,146.0
2,24.0,5407.0,140.0,2162.0,144.0
3,24.0,5437.0,139.0,2144.0,143.0
4,24.0,5438.0,138.0,2188.0,143.0


In [7]:
# Splitting up the data for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=42)

In [8]:
# Testing accuracy on training and testing data
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9337177776617112

## Making a prediction on test data set

In [9]:
# Making prediction based on test data
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
prediction = rf.predict_proba(X_test)
print(prediction)

[[1.   0.  ]
 [0.57 0.43]
 [0.99 0.01]
 ...
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]]


In [10]:
# Looking at important of each feature
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.40927773325089745, 'Temperature Seasonality'),
 (0.27394188104729605, 'Annual Precipitation'),
 (0.1439550162356342, 'Precipitation of Warmest Quarter'),
 (0.10502098405353673, 'Precipitation of Driest Quarter'),
 (0.06780438541263561, 'Precipitation of Driest Month')]

## Importing and cleaning new data set

In [23]:
mild_2070 = pd.read_csv('../../Resources/2070_mild.csv')

In [24]:
mild_2070.drop(['Unnamed: 0'], axis = 1, inplace = True)
mild_2070.dropna(axis = 0, how = 'any', inplace = True)
mild_2070.rename(columns={'bc45bi701': 'Annual Mean Temperature', 
                     'bc45bi7010': 'Mean Temperature of Warmest Quarter',
                     'bc45bi7011': 'Mean Temperature of Coldest Quarter',
                     'bc45bi7012': 'Annual Precipitation',
                     'bc45bi7013': 'Precipitation of Wettest Month',
                     'bc45bi7014': 'Precipitation of Driest Month',
                     'bc45bi7015': 'Precipitation Seasonality (Coefficient of Variation)',
                     'bc45bi7016': 'Precipitation of Wettest Quarter',
                     'bc45bi7017': 'Precipitation of Driest Quarter',
                     'bc45bi7018': 'Precipitation of Warmest Quarter',
                     'bc45bi7019': 'Precipitation of Coldest Quarter',
                     'bc45bi702': 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
                     'bc45bi703': 'Isothermality',
                     'bc45bi704': 'Temperature Seasonality',
                     'bc45bi705': 'Max Temperature of Warmest Month',
                     'bc45bi706': 'Min Temperature of Coldest Month',
                     'bc45bi707': 'Temperature Annual Range',
                     'bc45bi708': 'Mean Temperature of Wettest Quarter',
                     'bc45bi709': 'Mean Temperature of Driest Quarter',
                    }, inplace=True)


mild_2070.head()

,x,y,Annual Mean Temperature,Mean Temperature of Warmest Quarter,Mean Temperature of Coldest Quarter,Annual Precipitation,Precipitation of Wettest Month,Precipitation of Driest Month,Precipitation Seasonality (Coefficient of Variation),Precipitation of Wettest Quarter,...,Precipitation of Warmest Quarter,Precipitation of Coldest Quarter,Mean Diurnal Range (Mean of monthly (max temp - min temp)),Isothermality,Temperature Seasonality,Max Temperature of Warmest Month,Min Temperature of Coldest Month,Temperature Annual Range,Mean Temperature of Wettest Quarter,Mean Temperature of Driest Quarter
0,-124.779167,49.3375,105,171,49,1891,338,22,66,897,...,131,829,86,37,4912,248,19,229,54,171
1,-124.770833,49.3375,102,167,48,1909,340,23,65,903,...,135,834,85,37,4834,243,17,226,53,167
2,-124.762500,49.3375,101,166,46,1913,341,23,65,904,...,136,836,85,37,4832,241,15,226,52,166
3,-124.754167,49.3375,108,174,52,1856,332,22,67,883,...,127,814,86,37,4931,251,21,230,57,174
4,-124.745833,49.3375,115,183,56,1815,324,20,67,866,...,118,801,89,38,5062,261,27,234,61,183


In [25]:
x_new = mild_2070[['Precipitation of Driest Month', 'Temperature Seasonality', 'Precipitation of Driest Quarter',
             'Annual Precipitation', 'Precipitation of Warmest Quarter']]
x_new.head()

,Precipitation of Driest Month,Temperature Seasonality,Precipitation of Driest Quarter,Annual Precipitation,Precipitation of Warmest Quarter
0,22,4912,131,1891,131
1,23,4834,135,1909,135
2,23,4832,136,1913,136
3,22,4931,127,1856,127
4,20,5062,118,1815,118


## Making a prediction on our new data

In [26]:
mild2070_prediction = rf.predict_proba(x_new)

## Creating final dataframe and exporting

In [30]:
mild2070_final = mild_2070[['x', 'y']]
mild2070_final["Prediction"] = mild2070_prediction[:,1]
mild2070_final.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,x,y,Prediction
0,-124.779167,49.3375,0.0
1,-124.770833,49.3375,0.0
2,-124.762500,49.3375,0.0
3,-124.754167,49.3375,0.0
4,-124.745833,49.3375,0.0


In [31]:
mild2070_final.to_csv('2070_moderate_final.csv')